In [18]:
import os
import importlib

from openai import OpenAI

from regulations_rag.data_in_dataframes import load_parquet_data, save_parquet_data
from regulations_rag.embeddings import get_ada_embedding, get_closest_nodes

import cemad_rag.cemad_data
importlib.reload(cemad_rag.cemad_data) 
from cemad_rag.cemad_data import CEMADData


openai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"),)
key = os.getenv('excon_encryption_key')


In [2]:
bop = load_parquet_data("./inputs/bopcodes.parquet")

In [7]:
#bop.columns = bop.columns.str.lower()
#bop["embedding"] = ""
bop

,category,sub-category,category description,section,inward or outward,embedding
0,100,,Adjustments / Reversals / Refunds applicable t...,Transaction adjustments,Inward,
1,101,01,Export advance payment (excluding capital good...,Exports : Advance payments,Inward,
2,101,02,Export advance payment - capital goods,Exports : Advance payments,Inward,
3,101,03,Export advance payment – gold,Exports : Advance payments,Inward,
4,101,04,Export advance payment – platinum,Exports : Advance payments,Inward,
...,...,...,...,...,...,...
631,832,,Not allocated,Miscellaneous payments,Outwards,
632,833,,Credit/Debit card company settlement as well a...,Miscellaneous payments,Outwards,
633,834,,Not allocated,Miscellaneous payments,Outwards,
634,835,,Not allocated,Miscellaneous payments,Outwards,


In [12]:
embedding_model =  "text-embedding-3-large"
embedding_dimensions = 1024

increment = 10
for i in range(0, len(bop), increment):
    chunk = bop.iloc[i:i+increment].copy()
    chunk["embedding"] = chunk["category description"].apply(lambda x: get_ada_embedding(openai_client=openai_client, text=x, model=embedding_model, dimensions=embedding_dimensions))
    bop.loc[chunk.index, "embedding"] = chunk["embedding"]
    print(f"Completed {i+increment} lines")


Completed 10 lines
Completed 20 lines
Completed 30 lines
Completed 40 lines
Completed 50 lines
Completed 60 lines
Completed 70 lines
Completed 80 lines
Completed 90 lines
Completed 100 lines
Completed 110 lines
Completed 120 lines
Completed 130 lines
Completed 140 lines
Completed 150 lines
Completed 160 lines
Completed 170 lines
Completed 180 lines
Completed 190 lines
Completed 200 lines
Completed 210 lines
Completed 220 lines
Completed 230 lines
Completed 240 lines
Completed 250 lines
Completed 260 lines
Completed 270 lines
Completed 280 lines
Completed 290 lines
Completed 300 lines
Completed 310 lines
Completed 320 lines
Completed 330 lines
Completed 340 lines
Completed 350 lines
Completed 360 lines
Completed 370 lines
Completed 380 lines
Completed 390 lines
Completed 400 lines
Completed 410 lines
Completed 420 lines
Completed 430 lines
Completed 440 lines
Completed 450 lines
Completed 460 lines
Completed 470 lines
Completed 480 lines
Completed 490 lines
Completed 500 lines
Completed

In [13]:
save_parquet_data(bop, "./inputs/bopcodes.parquet")

In [14]:
bop

,category,sub-category,category description,section,inward or outward,embedding
0,100,,Adjustments / Reversals / Refunds applicable t...,Transaction adjustments,Inward,"[-0.054617300629615784, 0.04683411121368408, -..."
1,101,01,Export advance payment (excluding capital good...,Exports : Advance payments,Inward,"[0.03178254887461662, 0.07286100834608078, -0...."
2,101,02,Export advance payment - capital goods,Exports : Advance payments,Inward,"[-0.013348566368222237, 0.0890803337097168, -0..."
3,101,03,Export advance payment – gold,Exports : Advance payments,Inward,"[-0.028368012979626656, 0.038180239498615265, ..."
4,101,04,Export advance payment – platinum,Exports : Advance payments,Inward,"[-0.037426818162202835, 0.036047808825969696, ..."
...,...,...,...,...,...,...
631,832,,Not allocated,Miscellaneous payments,Outwards,"[-0.016109507530927658, 0.01816733554005623, 0..."
632,833,,Credit/Debit card company settlement as well a...,Miscellaneous payments,Outwards,"[-0.026186203584074974, 0.020568517968058586, ..."
633,834,,Not allocated,Miscellaneous payments,Outwards,"[-0.016109507530927658, 0.01816733554005623, 0..."
634,835,,Not allocated,Miscellaneous payments,Outwards,"[-0.016109507530927658, 0.01816733554005623, 0..."


In [25]:
q = "vat"
qe = get_ada_embedding(openai_client=openai_client, text=q, model=embedding_model, dimensions=embedding_dimensions)
search_results = get_closest_nodes(bop, "embedding", qe, threshold = 1.0)
search_results.nsmallest(16, 'cosine_distance') 

,category,sub-category,category description,section,inward or outward,embedding,cosine_distance
464,308,,Rental,Income payments,Outwards,"[-0.04304356873035431, 0.00858790148049593, -0...",0.683011
145,308,,Rental,Income receipts,Inward,"[-0.04304356873035431, 0.00858790148049593, -0...",0.683011
312,832,,Rand drafts/cheques drawn on vostro accounts (...,Miscellaneous payments,Inward,"[-0.010449634864926338, 0.05393928661942482, -...",0.694307
630,831,,Rand collections for the credit of vostro acco...,Miscellaneous payments,Outwards,"[-0.03091721050441265, 0.06293369829654694, -0...",0.714304
500,411,02,Tax - VAT refunds,Current payments,Outwards,"[-0.016300121322274208, 0.0633414089679718, -0...",0.732185
181,411,02,Tax - VAT refunds,Current payments,Inward,"[-0.016293389722704887, 0.06335949897766113, -...",0.732279
499,411,01,Tax - Income tax,Current payments,Outwards,"[-0.019195839762687683, 0.05866285413503647, -...",0.739425
180,411,01,Tax - Income tax,Current payments,Inward,"[-0.019195839762687683, 0.05866285413503647, -...",0.739425
285,701,01,Options – listed,Derivatives (excluding inward listed),Inward,"[-0.012603546492755413, -0.004046157002449036,...",0.747071
604,701,01,Options – listed,Derivatives (excluding inward listed),Outwards,"[-0.012603546492755413, -0.004046157002449036,...",0.747071
